In [ ]:
"""
extract genes sequences from reference genome, annotation 
Genes must have at least one mapped read (count) in any experiment.
Outputs a list of fasta genes, one for svevo and one for zavitan
"""

In [131]:
from biopyutils import geneGFF2FA
import pandas as pd
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq

In [132]:
#RUN only to fix tsv
df = pd.read_csv('disk/counts/svevo.counts.tsv', sep='\t', decimal=',')
df.to_csv('disk/counts/svevo.counts.csv', sep=',', index=None)

In [133]:
#SV
file_csv = 'disk/counts/svevo.counts.csv'
file_out = 'disk/counts/svevo.counts.fa'
file_fasta = 'disk/svevo3A.fasta'
file_annotation = 'disk/PGSB_TRITD_Jan2017_all.gff3'
file_intermediate = 'disk/counts/svevo.genes.csv'
bp_from = 54100000
bp_to = 78700000

In [125]:
#ZV
file_csv = 'disk/counts/zav_v2.counts.csv'
file_out = 'disk/counts/zav_v2.counts.fa'
file_fasta = 'disk/zavitan_3A.fasta'
file_annotation = 'disk/SORTED_WEW_v2_HC_e_LC_GFF3_CATTATI_gff_PAO_26feb18.gff3'
file_intermediate = 'disk/counts/zavitan_v2.genes.csv'
bp_from = 54600000
bp_to = 79100000

In [134]:
df = pd.read_csv(file_csv, sep=',')
print("Total: ",len(df.index))
df = df[(df['Chromosome.scaffold.name'] == 'chr3A') & (df['Gene.start..bp.'] >= bp_from ) & (df['Gene.end..bp.'] <= bp_to )]
print("Total segment: ",len(df.index))
df = df[(df.TL1 > 0) | (df.TL2 > 0) | (df.TL3 > 0)| (df.TD1 > 0)| (df.TD2 > 0) | (df.TD3 > 0)]
print("Lectures > 0: ",len(df.index))
df.to_csv(file_intermediate,sep=',', index=None)

Total:  369963
Total segment:  1019
Lectures > 0:  239


In [105]:
gene_list = df.id.tolist()
gene_list[:5]

['TRIDC3Av2G023650',
 'TRIDC3Av2G023690',
 'TRIDC3Av2G023850',
 'TRIDC3Av2G023880',
 'TRIDC3Av2G023890']

In [106]:
df_gff = pd.read_csv(file_annotation, index_col=False, sep='\t', header=None)
df_gff.columns = ['seqname', 'source', 'feature', 'start', 'end', 'score', 'strand', 'frame', 'attribute']
fasta_seq = SeqIO.parse(file_fasta, 'fasta')
df_gff = df_gff[(df_gff['seqname'] == 'chr3A') & (df_gff['start'] >= bp_from ) & (df_gff['end'] <= bp_to )]
df_gff.head(3)
fasta_seq = SeqIO.parse(file_fasta, 'fasta')
for record in fasta_seq:
    clean_seq = ''.join(str(record.seq).splitlines())

In [107]:
buffer_seqs = []
cont = 0
for gene in gene_list:
    print(gene)
    df_extract = df_gff[(df_gff.attribute.str.contains('ID='+gene)) & (df_gff.feature=='gene')]
    gene_gff = df_extract.iloc[0]
    if int(gene_gff.start) < 0:
        start = 0
    else:
        start = int(gene_gff.start)
    if int(gene_gff.end) > len(clean_seq):
        end = len(clean_seq)
    else:
        end = int(gene_gff.end)
    new_seq = clean_seq[start:end]
    att = gene_gff.attribute
    desc = "seq:" + str(record.id)
    desc += " start:" + str(gene_gff.start)
    desc += " end:" + str(gene_gff.end)
    desc += " strand:" + str(gene_gff.strand)
    seq = SeqRecord(Seq(new_seq), id=gene, description=desc)
    buffer_seqs.append(seq)


TRIDC3Av2G023650
TRIDC3Av2G023690
TRIDC3Av2G023850
TRIDC3Av2G023880
TRIDC3Av2G023890
TRIDC3Av2G023900
TRIDC3Av2G023980
TRIDC3Av2G023990
TRIDC3Av2G024010
TRIDC3Av2G024020
TRIDC3Av2G024100
TRIDC3Av2G024120
TRIDC3Av2G024130
TRIDC3Av2G024140
TRIDC3Av2G024160
TRIDC3Av2G024200
TRIDC3Av2G024210
TRIDC3Av2G024220
TRIDC3Av2G024230
TRIDC3Av2G024250
TRIDC3Av2G024260
TRIDC3Av2G024270
TRIDC3Av2G024280
TRIDC3Av2G024290
TRIDC3Av2G024400
TRIDC3Av2G024490
TRIDC3Av2G024530
TRIDC3Av2G024540
TRIDC3Av2G024550
TRIDC3Av2G024560
TRIDC3Av2G024570
TRIDC3Av2G024630
TRIDC3Av2G024650
TRIDC3Av2G024670
TRIDC3Av2G024680
TRIDC3Av2G024690
TRIDC3Av2G024930
TRIDC3Av2G024940
TRIDC3Av2G024950
TRIDC3Av2G024960
TRIDC3Av2G024970
TRIDC3Av2G025020
TRIDC3Av2G025110
TRIDC3Av2G025120
TRIDC3Av2G025130
TRIDC3Av2G025150
TRIDC3Av2G025200
TRIDC3Av2G025250
TRIDC3Av2G025260
TRIDC3Av2G025270
TRIDC3Av2G025280
TRIDC3Av2G025290
TRIDC3Av2G025410
TRIDC3Av2G025480
TRIDC3Av2G025490
TRIDC3Av2G025560
TRIDC3Av2G025570
TRIDC3Av2G025580
TRIDC3Av2G0255

In [108]:
SeqIO.write(buffer_seqs, file_out, "fasta")

249

746673839